In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
import matplotlib.pyplot as plt


In [2]:
!pip install prince

In [3]:
import prince

In [4]:
df = pd.read_csv('Hospital_Inpatient_Discharges_17_18_cleaned.csv')
df.head()

/var/folders/z6/k1gp26316czbm7zy5_d3bjvm0000gn/T/ipykernel_34871/4250625227.py:1: DtypeWarning: Columns (15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Hospital_Inpatient_Discharges_17_18_cleaned.csv')


,Unnamed: 0,year,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,...,APR MDC Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Emergency Department Indicator,Total Costs,Total Charges,Length of Stay
0,0,2017,Hudson Valley,Westchester,5903001.0,1061.0,Montefiore Mount Vernon Hospital,30 to 49,NaN,Male,...,Mental Diseases And Disorders,2.0,Moderate,Minor,Medical,Medicare,0,40464.04,114168.00,21.0
1,1,2017,Hudson Valley,Westchester,5903001.0,1061.0,Montefiore Mount Vernon Hospital,50 to 69,105,Male,...,Diseases And Disorders Of The Circulatory System,3.0,Major,Moderate,Medical,Medicaid,0,26046.50,58517.00,8.0
2,2,2017,Hudson Valley,Westchester,5903001.0,1061.0,Montefiore Mount Vernon Hospital,30 to 49,105,Female,...,Diseases And Disorders Of The Kidney And Urina...,2.0,Moderate,Moderate,Medical,Department of Corrections,0,17397.59,39647.04,6.0
3,3,2017,Hudson Valley,Westchester,5903001.0,1061.0,Montefiore Mount Vernon Hospital,50 to 69,105,Female,...,"Infectious And Parasitic Diseases, Systemic Or...",2.0,Moderate,Moderate,Medical,Medicaid,0,16118.84,36031.09,4.0
4,4,2017,Hudson Valley,Westchester,5903001.0,1061.0,Montefiore Mount Vernon Hospital,18 to 29,105,Female,...,"Infectious And Parasitic Diseases, Systemic Or...",2.0,Moderate,Minor,Medical,Department of Corrections,0,11921.05,30534.00,4.0


In [12]:
df_clean = df.drop(['Unnamed: 0', 'year', 'Hospital Service Area', 'Hospital County',
                    'Operating Certificate Number', 'Permanent Facility Id',
                    'Facility Name', 'Age Group', 'Zip Code - 3 digits', 'Gender', 'Race',
                    'Ethnicity','CCSR Procedure Code', 'CCSR Procedure Description',
                    'APR Medical Surgical Description', 'Payment Typology 1'], axis=1)
df_clean.dropna(inplace=True)
df_clean

,Type of Admission,Patient Disposition,Discharge Year,CCSR Diagnosis Code,CCSR Diagnosis Description,APR DRG Code,APR DRG Description,APR MDC Code,APR MDC Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,Emergency Department Indicator,Total Costs,Total Charges,Length of Stay
0,Elective,Home or Self Care,2017,659,Schizophrenia And Other Psychotic Disorders,750.0,Schizophrenia,19.0,Mental Diseases And Disorders,2.0,Moderate,Minor,0,40464.04,114168.00,21.0
1,Emergency,Skilled Nursing Home,2017,99,Hypertension With Complications And Secondary ...,194.0,Heart Failure,5.0,Diseases And Disorders Of The Circulatory System,3.0,Major,Moderate,0,26046.50,58517.00,8.0
2,Emergency,Court/Law Enforcement,2017,161,Other Diseases Of Kidney And Ureters,465.0,Urinary Stones & Acquired Upper Urinary Tract ...,11.0,Diseases And Disorders Of The Kidney And Urina...,2.0,Moderate,Moderate,0,17397.59,39647.04,6.0
3,Emergency,Skilled Nursing Home,2017,238,Complications Of Surgical Procedures Or Medica...,721.0,"Post-Operative, Post-Traumatic, Other Device I...",18.0,"Infectious And Parasitic Diseases, Systemic Or...",2.0,Moderate,Moderate,0,16118.84,36031.09,4.0
4,Emergency,Court/Law Enforcement,2017,2,Septicemia (Except In Labor),720.0,Septicemia & Disseminated Infections,18.0,"Infectious And Parasitic Diseases, Systemic Or...",2.0,Moderate,Minor,0,11921.05,30534.00,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4692959,Emergency,Left Against Medical Advice,2018,MBD018,Opioid-Related Disorders,770.0,"Drug & Alcohol Abuse Or Dependence, Left Again...",20.0,Alcohol/Drug Use And Alcohol/Drug Induced Orga...,1.0,Minor,Minor,0,1722.84,6764.00,2.0
4692960,Emergency,Home or Self Care,2018,INF002,Septicemia,720.0,Septicemia & Disseminated Infections,18.0,Infectious And Parasitic Diseases (Systemic Or...,4.0,Extreme,Major,1,11414.60,43488.79,7.0
4692961,Emergency,Home or Self Care,2018,GEN001,Nephritis; Nephrosis; Renal Sclerosis,462.0,Nephritis & Nephrosis,11.0,Diseases And Disorders Of The Kidney And Urina...,2.0,Moderate,Moderate,1,19997.25,86509.76,11.0
4692962,Emergency,Home or Self Care,2018,PRG022,Prolonged Pregnancy,560.0,Vaginal Delivery,14.0,"Pregnancy, Childbirth And The Puerperium",1.0,Minor,Minor,1,11657.60,21954.93,2.0


In [6]:
df_clean['APR DRG Description'].unique()

array(['Schizophrenia', 'Heart Failure',
       'Urinary Stones & Acquired Upper Urinary Tract Obstruction',
       'Post-Operative, Post-Traumatic, Other Device Infections',
       'Septicemia & Disseminated Infections', 'Alcoholic Liver Disease',
       'Chest Pain', 'Diabetes', 'Syncope & Collapse',
       'Infections Of Upper Respiratory Tract', 'Other Pneumonia',
       'Seizure', 'Cesarean Delivery', 'Acute Myocardial Infarction',
       'Chronic Kidney Disease',
       'Shoulder, Upper Arm & Forearm Procedures Except Joint Replacement',
       'Other & Unspecified Gastrointestinal Hemorrhage',
       'Neonate Birthwt >2499G, Normal Newborn Or Neonate W Other Problem',
       'Other Musculoskeletal System & Connective Tissue Diagnoses',
       'Vaginal Delivery',
       'Fractures & Dislocations Except Femur, Pelvis & Back',
       'Fracture Of Pelvis Or Dislocation Of Hip',
       'Malignancy Of Hepatobiliary System & Pancreas',
       'Lymphatic & Other Malignancies & Neoplasms

In [7]:
df_clean.columns

Index(['Type of Admission', 'Patient Disposition', 'Discharge Year',
       'CCSR Diagnosis Code', 'CCSR Diagnosis Description', 'APR DRG Code',
       'APR DRG Description', 'APR MDC Code', 'APR MDC Description',
       'APR Severity of Illness Code', 'APR Severity of Illness Description',
       'APR Risk of Mortality', 'Emergency Department Indicator',
       'Total Costs', 'Total Charges', 'Length of Stay'],
      dtype='object')

In [17]:
df_sample = df_clean.sample(100000)
df_sample.dropna(inplace=True)
y = df_sample[['Total Costs']]
# 12/06 updated to include Length of Stay as a feature to get better r2
X = df_sample.loc[:, ~df_clean.columns.isin(['Total Costs', 'Total Charges'])] 

In [18]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

# Handle categorical Columns
categorical_columns = X_train.select_dtypes(exclude='number')

for column in categorical_columns:
    values = df_sample[column].unique()
    encoded_values = [i for i in range(1, len(values) + 1)]
    X_train[column].replace(values, encoded_values, inplace=True)
    
estimator = DecisionTreeRegressor(criterion="squared_error", max_depth=10)
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X_train, y_train)

selector.ranking_

X_train.columns[np.where(selector.ranking_ == 1)]


Index(['Type of Admission', 'CCSR Diagnosis Code', 'APR DRG Code',
       'APR DRG Description', 'Length of Stay'],
      dtype='object')

In [20]:
for column in categorical_columns:
    values = df_sample[column].unique()
    encoded_values = [i for i in range(1, len(values) + 1)]
    X_test[column].replace(values, encoded_values, inplace=True)


dt = DecisionTreeRegressor(criterion="squared_error", max_depth=10)
dt.fit(X_train, y_train)
yhat = dt.predict(X_test)

mae = metrics.mean_absolute_error(y_test, yhat)
mse = metrics.mean_squared_error(y_test, yhat)
r2 = metrics.r2_score(y_test, yhat)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))


The model performance for testing set
--------------------------------------
MAE is 7037.3440385635595
MSE is 1028872597.3813533
R2 score is -0.3327676039607568


In [21]:
from sklearn.ensemble import RandomForestRegressor


X_train = X_train[X_train.columns[np.where(selector.ranking_ == 1)]]
X_test = X_test[X_test.columns[np.where(selector.ranking_ == 1)]]

y_train['Total Costs'] = y_train['Total Costs'].astype('float64')
y_train['Total Costs'] = np.log(y_train['Total Costs'].values.reshape(-1,1))

#print(y_train_all_features)

y_test['Total Costs'] = y_test['Total Costs'].astype('float64')
y_test['Total Costs'] = np.log(y_test['Total Costs'].values.reshape(-1,1))
rf = RandomForestRegressor(max_depth=2, random_state=42)

rf.fit(X_train, y_train)

preds = rf.predict(X_test)
print(preds)
print(y_test)
mae = metrics.mean_absolute_error(y_test, preds)
mse = metrics.mean_squared_error(y_test, preds)
r2 = metrics.r2_score(y_test, preds)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

/var/folders/z6/k1gp26316czbm7zy5_d3bjvm0000gn/T/ipykernel_34871/964374862.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


[9.71376422 8.86730425 9.71376422 ... 8.86730425 7.62671554 9.71376422]
         Total Costs
64007      10.147016
1150405    10.733472
3662824     9.444181
4613054    10.618719
530591      7.987491
...              ...
1028547    10.528959
55253       9.785651
2536404     9.072800
1006478     7.837767
2111284     8.996784

[25000 rows x 1 columns]
The model performance for testing set
--------------------------------------
MAE is 0.5630236198997128
MSE is 0.5227826209361447
R2 score is 0.5000866147828567


In [22]:
sch_df = df_clean[df_clean['APR DRG Code'] == 750].reset_index(drop=True)

sch_df
y = sch_df[['Total Costs']]
X = sch_df.loc[:, ~sch_df.columns.isin(['Total Costs', 'Total Charges'])]

In [23]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
categorical_columns = X_train.select_dtypes(exclude='number')

for column in categorical_columns:
    values = sch_df[column].unique()
    encoded_values = [i for i in range(1, len(values) + 1)]
    X_train[column].replace(values, encoded_values, inplace=True)
    X_test[column].replace(values, encoded_values, inplace=True)

lr = LinearRegression()
lr.fit(X_train, y_train)
yhat = lr.predict(X_test)

mae = metrics.mean_absolute_error(y_test, yhat)
mse = metrics.mean_squared_error(y_test, yhat)
r2 = metrics.r2_score(y_test, yhat)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

The model performance for testing set
--------------------------------------
MAE is 5022.983660477453
MSE is 86442358.68893988
R2 score is 0.7974103236992665


In [25]:
dt = DecisionTreeRegressor(criterion="squared_error", max_depth=10)
dt.fit(X_train, y_train)
yhat = dt.predict(X_test)

mae = metrics.mean_absolute_error(y_test, yhat)
mse = metrics.mean_squared_error(y_test, yhat)
r2 = metrics.r2_score(y_test, yhat)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))


tt = TransformedTargetRegressor(regressor=LinearRegression(), transformer=MinMaxScaler())
tt.fit(X_train, y_train)
tt.score(X_test, y_test)


The model performance for testing set
--------------------------------------
MAE is 5051.524567903435
MSE is 98798126.15228637
R2 score is 0.768452866165582


0.7974084721564288